In [1]:
!pip install --quiet --upgrade google_cloud_firestore google_cloud_aiplatform langchain langchain-google-vertexai langchain_community langchain_experimental pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.8/336.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5

In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel

import pickle
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

from google.cloud import firestore
from google.cloud.firestore_v1.vector import Vector
from google.cloud.firestore_v1.base_vector_query import DistanceMeasure

In [2]:
import vertexai
from google.cloud import aiplatform

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "qwiklabs-gcp-02-6db37dc5a410" # @param {type:"string"}

vertexai.init(project=PROJECT_ID, location=LOCATION)
print(f"Vertex AI SDK version: {aiplatform.__version__}")

Vertex AI SDK version: 1.70.0


In [14]:
# Initialize the a specific Embeddings Model version
embeddings = VertexAIEmbeddings("text-embedding-004")

In [4]:
!gcloud storage cp gs://partner-genai-bucket/genai069/nyc_food_safety_manual.pdf .

Copying gs://partner-genai-bucket/genai069/nyc_food_safety_manual.pdf to file://./nyc_food_safety_manual.pdf

Average throughput: 75.6MiB/s


In [32]:
loader = PyMuPDFLoader("./nyc_food_safety_manual.pdf")
data = loader.load()
data[0]

Document(metadata={'source': './nyc_food_safety_manual.pdf', 'file_path': './nyc_food_safety_manual.pdf', 'page': 0, 'total_pages': 94, 'format': 'PDF 1.6', 'title': 'FOR BIND Food Protect Manual rev6 14_Conv-Sig', 'author': 'Hizzoner', 'subject': '', 'keywords': '', 'creator': 'QuarkXPress 8.5', 'producer': 'Acrobat Distiller 8.0.0 (Macintosh)', 'creationDate': "D:20140624124242-04'00'", 'modDate': "D:20151112105727-05'00'", 'trapped': ''}, page_content='The Health Code\nThese are regulations that were\nformulated to allow the  Department\nto effectively protect the health of the\npopulation. Among the rules\nembodied in the Health Code is\nArticle 81 which regulates the oper-\nations of food establishments for\nthe purpose of preventing public\nhealth hazards.\nEnvironmental Health Division \nThe Division of Environmental\nHealth is the Commission within the\nHealth Department that is concerned\nwith public health and works to\neliminate the incidence of injury and\nillness caused by

In [6]:
def clean_page(page):
  return page.page_content.replace("-\n","")\
                          .replace("\n"," ")\
                          .replace("\x02","")\
                          .replace("\x03","")\
                          .replace("fo d P R O T E C T I O N  T R A I N I N G  M A N U A L","")\
                          .replace("N E W  Y O R K  C I T Y  D E P A R T M E N T  O F  H E A L T H  &  M E N T A L  H Y G I E N E","")

In [37]:
cleaned_pages = [clean_page(page) for page in data]
cleaned_pages[:5]

['The Health Code These are regulations that were formulated to allow the  Department to effectively protect the health of the population. Among the rules embodied in the Health Code is Article 81 which regulates the operations of food establishments for the purpose of preventing public health hazards. Environmental Health Division  The Division of Environmental Health is the Commission within the Health Department that is concerned with public health and works to eliminate the incidence of injury and illness caused by environmental factors. There are several Offices and Bureaus within this division. One of these is the Bureau of Food Safety and Community Sanitation that has the responsibility for conducting inspections of food service and food processing establishments. These inspections are performed by Public Health Sanitarians. Anti-corruption Warning All Sanitarians have Department of Health and Mental Hygiene badges and identification cards which they must display whenever it is 

In [73]:
chunked_content = []

# Only process the first five pages
#for page_content in cleaned_pages[:5]:
    # Create a SemanticTextSplitter instance using your embedding model
text_splitter = SemanticChunker(embeddings)
    # Split the page content into chunks
chunks = text_splitter.create_documents(cleaned_pages[:5])
    # Extend the chunked_content list with the new chunks
chunked_content.extend(chunks)


# Inspect a few chunks
print(chunks[2].page_content)


If you don’t see a date that is convenient, check back as new course dates are added frequently. 1   INTRODUCTION T he New York City Department of Health and Mental Hygiene has the jurisdiction to regulate all matters affecting health in the city and to perform all those functions and operations that relate to the health of the people of the city. INTRODUCTION 


In [74]:
#Use the embedding_model to generate embeddings of the text chunks, saving them to a list called chunked_embeddings
chunked_embeddings = []
chunked_embeddings = embeddings.embed_query(chunk)
print(chunked_embeddings)


[0.02475065551698208, -0.00070410524494946, -0.005083796102553606, 0.0025818957947194576, -0.013331704773008823, -0.007198393810540438, 0.039864689111709595, 0.029253505170345306, 0.014930332079529762, -0.034707579761743546, 0.03945614770054817, 0.06387560814619064, 0.08196710050106049, -0.011799144558608532, 0.026967056095600128, -0.045428864657878876, 0.03221005201339722, 0.030114280059933662, -0.04923228919506073, 0.005659307818859816, 0.005269278772175312, 0.010837364941835403, 0.04602358862757683, -0.017749115824699402, -0.03156326711177826, -0.003451866563409567, -0.03526823967695236, -0.06972555816173553, -0.04156719893217087, -0.022138750180602074, 0.00939546711742878, 0.00866911094635725, -0.01252972986549139, -2.0102948838029988e-05, 0.0370931476354599, 0.03593934327363968, -0.03531743586063385, 0.07494760304689407, -0.026348041370511055, -0.03559640422463417, -0.04244842007756233, -0.016245681792497635, 0.00850803405046463, 0.04073077067732811, 0.040828123688697815, 0.006276

In [40]:
!gsutil cp gs://partner-genai-bucket/genai069/chunked_content.pkl .
!gsutil cp gs://partner-genai-bucket/genai069/chunked_embeddings.pkl .

chunked_content = pickle.load(open("chunked_content.pkl", "rb"))
chunked_embeddings = pickle.load(open("chunked_embeddings.pkl", "rb"))

Copying gs://partner-genai-bucket/genai069/chunked_content.pkl...
/ [1 files][280.7 KiB/280.7 KiB]                                                
Operation completed over 1 objects/280.7 KiB.                                    
Copying gs://partner-genai-bucket/genai069/chunked_embeddings.pkl...
/ [1 files][  1.8 MiB/  1.8 MiB]                                                
Operation completed over 1 objects/1.8 MiB.                                      


In [78]:
db = firestore.Client()
collection = db.collection("food-safety")

In [79]:
for i, chunk in enumerate(chunked_content):
    document_ref = collection.document(f"doc_{i}")
    document_ref.set({
        "content": chunk,
        "embedding": Vector(chunked_embeddings[i])
    })


TypeError: 'float' object is not iterable

In [80]:
from vertexai.language_models import TextEmbeddingModel

def text_embedding(text_to_embed) -> list:
    """Text embedding with a Large Language Model."""
    embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")
    embeddings = embedding_model.get_embeddings([text_to_embed])
    for embedding in embeddings:
        vector = embedding.values
        print(f"Length of Embedding Vector: {len(vector)}")
    return vector

In [76]:
from google.cloud.firestore_v1.base_vector_query import DistanceMeasure

def search_vector_database(query):
    """Search Firestore for documents similar to the query."""
    query_embedding = text_embedding(query)
    # Find the 5 nearest neighbors to the query embedding
    results = collection.find_nearest(
    vector_field="embedding",
    query_vector=Vector(query_embedding),
    distance_measure=DistanceMeasure.EUCLIDEAN,
    limit=5,
    )


    # Extract the document data from the search results
    snapshots = results.get()
    context = ""
    for snapshot in snapshots:
        context += snapshot.to_dict()['content']
    return context




In [77]:
#Test the search_firestore function with a query string.
search_vector_database("How should I store food?")

Length of Embedding Vector: 768


NameError: name 'collection' is not defined